## Read data using pandas

In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras
keras = tensorflow.keras

SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

csv_file = keras.utils.get_file(
    'heart.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/heart.csv')
    


In [2]:
df: pd.DataFrame = pd.read_csv(csv_file)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [3]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

In [4]:
target = df.pop('target')

In [5]:
numeric_feature_names = ['age', 'thalach', 'trestbps',  'chol', 'oldpeak']
numeric_features = df[numeric_feature_names]
numeric_features.head()

,age,thalach,trestbps,chol,oldpeak
0,63,150,145,233,2.3
1,67,108,160,286,1.5
2,67,129,120,229,2.6
3,37,187,130,250,3.5
4,41,172,130,204,1.4


In [6]:
tf.convert_to_tensor(numeric_features, dtype=tf.float32)

<tf.Tensor: shape=(303, 5), dtype=float32, numpy=
array([[ 63. , 150. , 145. , 233. ,   2.3],
       [ 67. , 108. , 160. , 286. ,   1.5],
       [ 67. , 129. , 120. , 229. ,   2.6],
       ...,
       [ 65. , 127. , 135. , 254. ,   2.8],
       [ 48. , 150. , 130. , 256. ,   0. ],
       [ 63. , 154. , 150. , 407. ,   4. ]], dtype=float32)>

In [7]:
normalizer = keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)

In [8]:
normalizer(numeric_features.loc[:3])

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[ 0.93383914,  0.03480718,  0.74578077, -0.26008663,  1.0680453 ],
       [ 1.3782105 , -1.7806165 ,  1.5923285 ,  0.7573877 ,  0.38022864],
       [ 1.3782105 , -0.87290466, -0.6651321 , -0.33687714,  1.3259765 ],
       [-1.9545751 ,  1.634109  , -0.10076694,  0.06627306,  2.0997703 ]],
      dtype=float32)>

In [9]:
from keras.models import Sequential
from keras.layers import Dense

def get_basic_model():
    model = Sequential([
        normalizer,
        Dense(10, activation='relu'),
        Dense(10, activation='relu'),
        Dense(1),
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
    return model

In [10]:
model = get_basic_model()
history = model.fit(numeric_features, target, epochs=15, batch_size=BATCH_SIZE)

Epoch 1/15
152/152 [==============================] - 3s 8ms/step - loss: 0.6749 - accuracy: 0.7228
Epoch 2/15
152/152 [==============================] - 1s 8ms/step - loss: 0.5414 - accuracy: 0.7261
Epoch 3/15
152/152 [==============================] - 1s 8ms/step - loss: 0.4802 - accuracy: 0.7426
Epoch 4/15
152/152 [==============================] - 1s 8ms/step - loss: 0.4511 - accuracy: 0.7525
Epoch 5/15
152/152 [==============================] - 1s 8ms/step - loss: 0.4379 - accuracy: 0.7558
Epoch 6/15
152/152 [==============================] - 1s 8ms/step - loss: 0.4293 - accuracy: 0.7756
Epoch 7/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4259 - accuracy: 0.7954
Epoch 8/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4239 - accuracy: 0.7855
Epoch 9/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4197 - accuracy: 0.7921
Epoch 10/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4179 - accuracy: 0.7987

## With tf.data

In [15]:
from tensorflow import data


numeric_dataset = data.Dataset.from_tensor_slices((numeric_features, target))


In [16]:
for row in numeric_dataset.take(3):
    print(row)

(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 63. , 150. , 145. , 233. ,   2.3])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 67. , 108. , 160. , 286. ,   1.5])>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 67. , 129. , 120. , 229. ,   2.6])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [17]:
numeric_batches = numeric_dataset.shuffle(1000).batch(BATCH_SIZE)

model = get_basic_model()
model.fit(numeric_batches, epochs=15)


Epoch 1/15
152/152 [==============================] - 2s 8ms/step - loss: 0.6608 - accuracy: 0.7195
Epoch 2/15
152/152 [==============================] - 1s 8ms/step - loss: 0.5672 - accuracy: 0.7261
Epoch 3/15
152/152 [==============================] - 1s 8ms/step - loss: 0.5120 - accuracy: 0.7261
Epoch 4/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4780 - accuracy: 0.7261
Epoch 5/15
152/152 [==============================] - 1s 8ms/step - loss: 0.4597 - accuracy: 0.7327
Epoch 6/15
152/152 [==============================] - 1s 8ms/step - loss: 0.4497 - accuracy: 0.7558
Epoch 7/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4422 - accuracy: 0.7558
Epoch 8/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4369 - accuracy: 0.7624
Epoch 9/15
152/152 [==============================] - 1s 9ms/step - loss: 0.4322 - accuracy: 0.7756
Epoch 10/15
152/152 [==============================] - 1s 9ms/step - loss: 0.4285 - accuracy: 0.7789

## DataFrame with heterogeneous data
use dataframe as dict

In [18]:
numeric_dict_ds = tf.data.Dataset.from_tensor_slices((dict(numeric_features), target))


In [19]:
for row in numeric_dict_ds.take(3):
  print(row)

({'age': <tf.Tensor: shape=(), dtype=int64, numpy=63>, 'thalach': <tf.Tensor: shape=(), dtype=int64, numpy=150>, 'trestbps': <tf.Tensor: shape=(), dtype=int64, numpy=145>, 'chol': <tf.Tensor: shape=(), dtype=int64, numpy=233>, 'oldpeak': <tf.Tensor: shape=(), dtype=float64, numpy=2.3>}, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
({'age': <tf.Tensor: shape=(), dtype=int64, numpy=67>, 'thalach': <tf.Tensor: shape=(), dtype=int64, numpy=108>, 'trestbps': <tf.Tensor: shape=(), dtype=int64, numpy=160>, 'chol': <tf.Tensor: shape=(), dtype=int64, numpy=286>, 'oldpeak': <tf.Tensor: shape=(), dtype=float64, numpy=1.5>}, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
({'age': <tf.Tensor: shape=(), dtype=int64, numpy=67>, 'thalach': <tf.Tensor: shape=(), dtype=int64, numpy=129>, 'trestbps': <tf.Tensor: shape=(), dtype=int64, numpy=120>, 'chol': <tf.Tensor: shape=(), dtype=int64, numpy=229>, 'oldpeak': <tf.Tensor: shape=(), dtype=float64, numpy=2.6>}, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
